In [1]:
from collections import OrderedDict
import math
import datetime
import time

import matplotlib.pyplot as plt
import pandas as pd
from IPython.core.display import HTML, display

import bokeh.io
import bokeh.plotting
import bokeh.models
import bokeh.palettes

import fto
from fto import stats


In [2]:
%load_ext autoreload
%autoreload 2
bokeh.io.output_notebook()

Loading BokehJS ...

In [3]:
class HTMLDict(OrderedDict):
    def _repr_html_(self):
        output_string_list = ["<table><tbody>"]
        for key, val in self.items():
            output_string_list.append('<tr><th>%s</th><td>%s</td></tr>' % (key, val))
        output_string_list.append("</tbody></table>")
        return ''.join(output_string_list)
            

In [4]:
df = fto.load_dataframe("/home/hugh/data/fto-stats.csv")
monthly_df = fto.stats.generate_monthly_dataframe(df)

In [5]:
display(HTML(monthly_df.to_html(index=False)))

Month,Births,Deaths,Pregnancies
March 2016,54,63,21
April 2016,68,44,29
May 2016,45,43,18
June 2016,58,68,28
July 2016,79,59,28
August 2016,37,67,16
September 2016,37,49,19


In [6]:
def generate_montly_figure(monthly_df):
    monthly_df.index = month_to_timestamp(monthly_df.Month)
    monthly_df.index.name = "TS"
    max_val = find_maximum_value(monthly_df)
    palette = bokeh.palettes.Set2_3
    source = bokeh.models.ColumnDataSource(monthly_df)
    fig = bokeh.plotting.figure(x_axis_type="datetime", width=800)
    x_axis = monthly_df.index.to_series()
    PROPS = {'line_width': 5, 'line_cap': 'round', 'source': source}
    #fig.line("TS", "Births", source=source)
    fig.line("TS", "Births", color=palette[0], legend="Births", **PROPS)
    fig.line("TS", "Deaths", color=palette[1], legend="Deaths", **PROPS)
    fig.line("TS", "Pregnancies", color=palette[2], legend="Pregnancies", **PROPS)
    fig.y_range = bokeh.models.Range1d(0, max_val, bounds="auto")
    fig.x_range = bokeh.models.Range1d(monthly_df.index.min(), monthly_df.index.max(), bounds="auto")
    fig.toolbar.logo = None
    configure_legend(fig.legend)  
    fig.yaxis.minor_tick_line_alpha = 0
    return fig


def find_maximum_value(monthly_df):
    max_series = monthly_df.max()
    max_val = max_series[max_series.apply(lambda val: not isinstance(val, str))].max()
    max_val_roundup = math.ceil(max_val/10) * 10
    return max_val_roundup

def month_to_timestamp(month_series):
    return month_series.apply(lambda val: pd.Timestamp.strptime(val, "%B %Y"))

def configure_legend(legend):
    legend.background_fill_alpha = 0
    legend.border_line_alpha = 0
    legend.spacing = 0
    legend.margin = 0
    legend.padding = 5

monthly_df = stats.generate_monthly_dataframe(df)
fig = generate_montly_figure(monthly_df)
bokeh.io.show(fig)

In [7]:
import bokeh.charts

In [8]:
records = stats.average_stats(df, monthly_df)
record_list = stats.RecordFormatList(records)
HTMLDict(record_list)

Average Birth Queue Time,3.8 months
Projected Birth Queue Time Entering Now,4.9 months
Average Number of Babies per Pregnancy,2.4 babies


In [9]:
from fto import fto_web

In [37]:
layout = fto_web.run(df)
bokeh.io.show(layout)

In [13]:
import bokeh.themes
import yaml
m = layout.children[1]

In [16]:
m

In [20]:
layout.apply_theme?

In [21]:
from bokeh.themes import Theme

In [68]:
bokeh.io.show(layout)